# Intro

In this notebook, I was planning to better understand how many spliceQTLs are due to SNPs in core splice sites (5'ss or 3'ss). As a simple first pass at this, I will look at the QTLtools permutation pass output in which I performed permutation pass on leafcutter intron phenotypes, using the smallest P-value amongst any intron within a cluster as the test statistic. Therefore, this QTLtools output has a single nominal P value (and single FDR corrected Qvalue) for each cluster of introns. So the way I plan to go about this, is to check for each cluster, check if any of the introns in the cluster overlap splice sites, then look for a pattern between the cluster P-value and whether any of the cluster's introns contain a splice site.

Now onto the code...

## import libs

Note that plotnine was not importing properly, so I just commented it out. This is the library I would normally use for plotting the results. Since it is working, I will just do some data processing to get me closer to the anser, write out the processed results to a file, then explore the results with plots in a different document with R where I am more comfortable plotting.

In [122]:
import os
import pandas as pd
import gzip
import vcf
# from plotnine import *

## helper functions

In [93]:
def count_iterable(i):
    return sum(1 for e in i)

def Count_SNPs_Around_Pos(vcf_reader, chrom, pos, upstream_bp, downstream_bp, strand="+"):
    """
    for a vcf_reader object, return an integer of the number of variants that intersect a window
    defined by a upstream_bp (normally negative), and a downstream_bp (normally positive),
    relative to a pos on a chrom. Reverse upstream_bp and downstream_bp if strand not +
    """
    if strand=="+":
        return count_iterable(vcf_reader.fetch(chrom, pos+upstream_bp, pos+downstream_bp))
    else:
        return count_iterable(vcf_reader.fetch(chrom, pos-downstream_bp, pos-upstream_bp))

def PickArg2IfStrandNegative(arg1, arg2, strand):
    if strand == "+":
        return arg1
    else:
        return arg2

## read clusters

Read introns from QTLtools bedgz input, which has intron coordinates and cluster ID for each intron. Then read in QTLtools permutation test output (permutation tested grouped by cluster) to get Q value for each cluster being a QTL.

In [4]:
splicing_bedgz_f = "../QTLs/QTLTools/polyA.Splicing/OnlyFirstReps.sorted.qqnorm.bed.gz"
Clusters = pd.read_csv(splicing_bedgz_f, delimiter='\t', usecols=range(0,5))


splicing_QTL_permutation_test_f = "../QTLs/QTLTools/polyA.Splicing/PermutationPass.FDR_Added.txt.gz"
Cluster_sig = pd.read_csv(splicing_QTL_permutation_test_f, delimiter=' ', usecols=['grp_id', 'q'])


Merged = pd.merge(Clusters, Cluster_sig, left_on="gid", right_on="grp_id", how="left")
Merged

,#Chr,start,end,pid,gid,grp_id,q
0,chr1,15947,16607,1:15947:16607:clu_1_-,chr1_clu_1_-,chr1_clu_1_-,0.51821
1,chr1,16310,16607,1:16310:16607:clu_1_-,chr1_clu_1_-,chr1_clu_1_-,0.51821
2,chr1,17055,17915,1:17055:17915:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293
3,chr1,17055,17233,1:17055:17233:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293
4,chr1,17368,17915,1:17368:17915:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293
...,...,...,...,...,...,...,...
143404,chr9,137798914,137800880,9:137798914:137800880:clu_20895_+,chr9_clu_20895_+,chr9_clu_20895_+,0.76432
143405,chr9,137800575,137800880,9:137800575:137800880:clu_20895_+,chr9_clu_20895_+,chr9_clu_20895_+,0.76432
143406,chr9,137800984,137802409,9:137800984:137802409:clu_20894_+,chr9_clu_20894_+,chr9_clu_20894_+,0.43322
143407,chr9,137800984,137802857,9:137800984:137802857:clu_20894_+,chr9_clu_20894_+,chr9_clu_20894_+,0.43322


Now, let's add a column for the donor position (5' intron boundary), and the acceptor (3' intron boundary), which will have to take strandedness into account.

In [110]:
Merged['strand']=Merged['pid'].str[-1:]
Merged['DonorPos']=Merged.apply(lambda x: PickArg2IfStrandNegative(x['start'], x['end']-1, x['strand']), axis=1)
Merged['AcceptorPos']=Merged.apply(lambda x: PickArg2IfStrandNegative(x['end']-1, x['start'], x['strand']), axis=1)

Merged

,#Chr,start,end,pid,gid,grp_id,q,DonorSnpCount,strand,DonorPos,AcceptorPos,AcceptorSnpCount
0,chr1,15947,16607,1:15947:16607:clu_1_-,chr1_clu_1_-,chr1_clu_1_-,0.51821,0,-,16606,15947,0
1,chr1,16310,16607,1:16310:16607:clu_1_-,chr1_clu_1_-,chr1_clu_1_-,0.51821,0,-,16606,16310,0
2,chr1,17055,17915,1:17055:17915:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293,0,-,17914,17055,0
3,chr1,17055,17233,1:17055:17233:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293,0,-,17232,17055,0
4,chr1,17368,17915,1:17368:17915:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293,0,-,17914,17368,0
...,...,...,...,...,...,...,...,...,...,...,...,...
143404,chr9,137798914,137800880,9:137798914:137800880:clu_20895_+,chr9_clu_20895_+,chr9_clu_20895_+,0.76432,0,+,137798914,137800879,0
143405,chr9,137800575,137800880,9:137800575:137800880:clu_20895_+,chr9_clu_20895_+,chr9_clu_20895_+,0.76432,0,+,137800575,137800879,0
143406,chr9,137800984,137802409,9:137800984:137802409:clu_20894_+,chr9_clu_20894_+,chr9_clu_20894_+,0.43322,0,+,137800984,137802408,0
143407,chr9,137800984,137802857,9:137800984:137802857:clu_20894_+,chr9_clu_20894_+,chr9_clu_20894_+,0.43322,0,+,137800984,137802856,0


Now, I want to classify the introns by whether or not there is a test SNP near the intron boundaries... I did some careful checking of my code with the help of visualizing SNPs and splice sites in IGV, to check that this counting works as expected with regards to both plus and minus strand introns at both 5'ss and 3'ss SNPs.

In [99]:
vcf_reader = vcf.Reader(filename='../QTLs/QTLTools/Expression.Splicing/Genotypes/WholeGenome.vcf.gz')

#From -1 to 5 pos relative to donor GT: [0]G[1]T
Merged['DonorSnpCount'] = Merged.apply(lambda x: Count_SNPs_Around_Pos(vcf_reader, x['#Chr'], x['DonorPos'], -1, 5, strand=x['strand']), axis=1)

,#Chr,start,end,pid,gid,grp_id,q,DonorSnpCount,strand,DonorPos,AcceptorPos
0,chr1,15947,16607,1:15947:16607:clu_1_-,chr1_clu_1_-,chr1_clu_1_-,0.51821,0,-,16606,15947
1,chr1,16310,16607,1:16310:16607:clu_1_-,chr1_clu_1_-,chr1_clu_1_-,0.51821,0,-,16606,16310
2,chr1,17055,17915,1:17055:17915:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293,0,-,17914,17055
3,chr1,17055,17233,1:17055:17233:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293,0,-,17232,17055
4,chr1,17368,17915,1:17368:17915:clu_2_-,chr1_clu_2_-,chr1_clu_2_-,0.79293,0,-,17914,17368
...,...,...,...,...,...,...,...,...,...,...,...
143404,chr9,137798914,137800880,9:137798914:137800880:clu_20895_+,chr9_clu_20895_+,chr9_clu_20895_+,0.76432,0,+,137798914,137800880
143405,chr9,137800575,137800880,9:137800575:137800880:clu_20895_+,chr9_clu_20895_+,chr9_clu_20895_+,0.76432,0,+,137800575,137800880
143406,chr9,137800984,137802409,9:137800984:137802409:clu_20894_+,chr9_clu_20894_+,chr9_clu_20894_+,0.43322,0,+,137800984,137802409
143407,chr9,137800984,137802857,9:137800984:137802857:clu_20894_+,chr9_clu_20894_+,chr9_clu_20894_+,0.43322,0,+,137800984,137802857


In [116]:
# From -3 position to 0 (YAG) 3'ss
Merged['AcceptorSnpCount'] = Merged.apply(lambda x: Count_SNPs_Around_Pos(vcf_reader, x['#Chr'], x['AcceptorPos'], -3, 0, strand=x['strand']), axis=1)


Now make a column to categorize introns as having SNP in splice site or not.

In [119]:
Merged['SNP_in_SS'] = (Merged['AcceptorSnpCount'] + Merged['DonorSnpCount']) > 0

How many introns SNPs overlapping the splice sites?

In [123]:
sum(Merged['SNP_in_SS'])

2982

Finally, let's write out the pertinent results, so I can read them into R in a different document and plot the results:

In [125]:
Merged.to_csv("../QTLs/QTLTools/polyA.Splicing/PermutationPass.FDR_Added.SS_SNPs.Annotated.txt.gz", sep="\t", index=False)